In [6]:
from selenium.webdriver.common.by import By
from scrapper import Scrappers
import random
import time
from selenium import webdriver
import os 
import pickle
from selenium.webdriver.common.keys import Keys
from models.database import Neo4jHandler

In [7]:
cookies_file = "cookies/facebook.pkl"
base_url = "https://www.facebook.com"

In [8]:
driver = webdriver.Chrome()
driver.maximize_window()

In [9]:
def save_cookies(filename, cookies):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "wb") as file:
        pickle.dump(cookies, file)

def load_cookies(filename):
    if os.path.exists(filename):
        with open(filename, "rb") as file:
            return pickle.load(file)
    return []


In [10]:
def login(username, password):
    try:
        driver.get("https://www.facebook.com/")
        email = driver.find_element(By.XPATH, "//input[@type='text']")
        passwd = driver.find_element(By.XPATH, "//input[@type='password']")
        email.send_keys(username)
        passwd.send_keys(password)
        submit = driver.find_element(By.XPATH, "//*[contains(text(), 'Log in')]")
        submit.click()
    except Exception as e:
        print("unable to complete login process with error ", e)

    try:
        save_div = driver.find_element(By.XPATH, "//div[@role='button' and .//*[contains(text(), 'Save')]]")
        save_div.click()
    except Exception as e:
        print("unable to perform click action on save button with error ", e)

    if driver.current_url == "https://www.facebook.com/settings":
        return driver.get_cookies()
    return None


In [20]:
def start_driver(username, password):
    driver.get(base_url)
    if os.path.exists(cookies_file):
        cookies = load_cookies(cookies_file)
        for cookie in cookies:
            try:
                driver.add_cookie(cookie)
            except Exception as e:
                pass
    else:
        if username and password:
            cookies = login(username, password)
            if cookies:
                save_cookies(cookies_file, cookies)

In [13]:
def scroll_followers(pause_time_range=(2, 5), scroll_amount=500):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
        time.sleep(random.uniform(*pause_time_range))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [16]:
def extract_profile(url):
    driver.get(url)
    result = {
        "profile_image": "",
        "profile_name": "",
        "profile_url": "",
        "profile_username": ""
    }
    try:
        profile_section = driver.find_element(By.XPATH, '//div[@class="x9f619 x1ja2u2z x78zum5 x2lah0s x1n2onr6 xl56j7k x1qjc9v5 xozqiw3 x1q0g3np x1l90r2v x1ve1bff"]')
        result["profile_image"] = profile_section.find_element(By.XPATH, ".//img").get_attribute("src")
        result["profile_name"] = profile_section.find_element(By.XPATH, ".//h1").text
        result["profile_url"] = driver.current_url
        result["profile_username"] = driver.current_url.split("/")[-1]
    except Exception as e:
        pass
    return result

In [17]:
def scroll_by_pressing_end():
    count = 0
    timer = 0
    wait_count = 20
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element("tag name", "body").send_keys(Keys.END)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if last_height >= new_height:
            count += 1
        else:
            count = 0
            
        if count >= 5:
            break

        last_height = new_height
        timer += 1
        
        if timer >= wait_count:
            timer = 0
            time.sleep(random.randint(5, 10))
        else:
            time.sleep(random.randint(0, 3))

In [28]:
def extract_friends(url):
    driver.get(url + "/friends")
    scroll_by_pressing_end()
    friends_div = driver.find_elements(By.XPATH, '//div[@class="x78zum5 x1q0g3np x1a02dak x1qughib"]/div')
    result = []
    for friend in friends_div:
        data = {}
        try:
            username = friend.find_element(By.XPATH, './/a[@tabindex="0"]/span')
            data["name"] = username.text
            data["profile_link"] = friend.find_element(By.XPATH, ".//a").get_attribute("href")
            data["username"] = data["profile_link"].split("/")[-1]
        except Exception:
            pass
        if not data:
            continue
        result.append(data)
    return result

In [29]:
neohandler = Neo4jHandler()
def start(url):
    start_driver(username="", password="")
    data = extract_profile(url)
    neohandler.create_person(profile_url=data.get("profile_url"), profile_name=data.get("profile_name"), profile_image=data.get("profile_image"), profile_username=data.get("profile_username"))
    res = extract_friends(url)
    for data in res:
        neohandler.create_friends("nina.visser.3", name=data.get("name"), profile_link=data.get("profile_link"), username=data.get("username"))

[+] Connected to Neo4j database: test


In [30]:
start("https://www.facebook.com/nina.visser.3")